In [ ]:
# Setup
import os
import pandas as pd
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt 

start_time = "2022-06-01 00:00"
end_time = "2022-06-01 05:00"

## Sources

* https://registry.opendata.aws/noaa-hrrr-pds/

* https://spire.com/tutorial/spire-weather-tutorial-intro-to-processing-grib2-data-with-python/

* https://github.com/microsoft/AIforEarthDataSets/blob/main/data/noaa-hrrr.md

* https://nbviewer.org/github/microsoft/AIforEarthDataSets/blob/main/data/noaa-hrrr.ipynb

* https://github.com/ecmwf/cfgrib/issues/63

* https://github.com/blaylockbk/Herbie/discussions/45

## Coordinates of Interest

Read in data frame of latitude and longitude coordinates that correspond to RAWS station locations.

In [ ]:
# Read in list of RAWS Stations
df = pd.read_csv("raws_stations_WA.csv")

# Filter to those with complete fmda data
df = df[(df[['air_temp', 'relative_humidity', 'precip_accum',
       'fuel_moisture', 'wind_speed', 'solar_radiation']]==1).sum(axis=1)==6]

# Get first 10 rows for demo
df = df[0:10]

# Get list of coords
points = list(df[["lon","lat"]].itertuples(index=False,name=None))
names = np.unique(df['STID'])

print(df.shape)

## DEMO: Download HRRR Grib

Given a single time slice, download the corresponding surface level HRRR grib file from AWS.

In [ ]:
from gather_HRRR import download_grib

In [ ]:
tempfile, url = download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com",
    time = end_time,
    model = "wrfsfcf",
    dest_dir =  "data" # destination subdirectory for url content
)

## Extract Atmospheric Data

Currently just gets temp and RH. Code modified from B. Blaylock: https://stackoverflow.com/questions/58758480/xarray-select-nearest-lat-lon-with-multi-dimension-coordinates

In [ ]:
ds=xr.open_dataset(
    tempfile,
    filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}
)

### Manually Extract and Plot

This section demonstrates how to extract atmospheric data and plots it to show nearest neighbors methodology (again from previous B. Blaylock source).

In [ ]:
## Function not used anymore

# def west_to_east(pts):
#     # Convert longitude in list of tuples 
#     # from deg W to deg E
#     # pts: list of tuples of form (lon, lat)

#     ## Extract list of lons and lats
#     lons = list(map(lambda pt: pt[0], pts))
#     lat = list(map(lambda pt: pt[1], pts))
#     ## Convert deg west to deg east
#     lons = list(map(lambda l: l+360,lons))
    
#     ## Combine back into list of tuples
#     coords = [(lons[i], lats[i]) for i in range(0, len(lons))]
    
#     return coords

In [ ]:
# Desired Lat/Lon pt
lon = points[0][0]
lat = points[0][1]
# Convert to degrees east if HRRR formatted that way
if ds.longitude.attrs['units']=="degrees_east":
    lon = 360 + lon
    # print(lon)
    
# Code modified from B. Blaylock https://stackoverflow.com/questions/58758480/xarray-select-nearest-lat-lon-with-multi-dimension-coordinates
ds.t2m.plot(x='longitude', y='latitude')

abslat = np.abs(ds.latitude-lat)
abslon = np.abs(ds.longitude-lon)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c))

# Now I can use that index location to get the values at the x/y diminsion
point_ds = ds.sel(x=yloc, y=xloc)

# Plot requested lat/lon point blue
plt.scatter(lon, lat, color='b')
plt.text(lon, lat, 'requested')

# Plot nearest point in the array red
g=plt.scatter(point_ds.longitude, point_ds.latitude, color='r')
g.set_facecolor('none')
plt.text(point_ds.longitude, point_ds.latitude, 'nearest')

plt.title('temp at nearest point: %s' % point_ds.t2m.data)

In [ ]:
# Desired Lat/Lon pt
lon = points[9][0]
lat = points[9][1]
# Convert to degrees east if HRRR formatted that way
if ds.longitude.attrs['units']=="degrees_east":
    lon = 360 + lon
    # print(lon)|
    
# Code modified from B. Blaylock https://stackoverflow.com/questions/58758480/xarray-select-nearest-lat-lon-with-multi-dimension-coordinates
ds.t2m.plot(x='longitude', y='latitude')

abslat = np.abs(ds.latitude-lat)
abslon = np.abs(ds.longitude-lon)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c))

# Now I can use that index location to get the values at the x/y diminsion
point_ds = ds.sel(x=yloc, y=xloc)

# Plot requested lat/lon point blue
plt.scatter(lon, lat, color='b')
plt.text(lon, lat, 'requested')

# Plot nearest point in the array red
g=plt.scatter(point_ds.longitude, point_ds.latitude, color='r')
g.set_facecolor('none')
plt.text(point_ds.longitude, point_ds.latitude, 'nearest')

plt.title('temp at nearest point: %s' % point_ds.t2m.data)

### Extract Atmospheric Data with Module Functions

In [ ]:
from gather_HRRR import extract_2m_vars

In [ ]:
## NOTE: compare values to previous plots
extract_2m_vars(ds, points[0]).t2m.values

In [ ]:
extract_2m_vars(ds, points[9]).t2m.values

---

## Build Timeseries by Looping

Get time range by looping through dates and: 

* temporarily downloading grib file at that date
* extract values at desired coordinates from temp file to build time series
* delete tempfile before iterating

In [ ]:
from gather_HRRR import gather_hrrr_time_range

In [ ]:
hrrr_dat = gather_hrrr_time_range(
    start = start_time,
    end = end_time,
    pts = points,
)

In [ ]:
# Compare to demo above
print(hrrr_dat[5,0,0])
print(hrrr_dat[5,9,0])